### Word to Vector

Word2vec (word embedding)词用向量表示。分skip-gram 和CBOW 两种训练方法。数据量大时，skip-gram效果好。
Skip-gram 基于中心词生成背景词，CBOW 基于背景词生成中心词。

#### Skip-gram

Skip-gram 思路是相邻越近的两个词的相似度越高。越远的相似度越低。

- 近义词：两个词是近义词，则这两个词分别邻近的词会很相似，所以近义词的词向量会很相似。

- 二次采样：对于出现频率高的词，比如介词the，所有的名词都可能跟the 邻近，但这些名词词向量肯定大同小异，因此去掉高频词，训练出的词向量效果更好。

- 负采样：求解中心词出现的条件下背景词出现的概率，需要两个词向量的内积，除以中心词和其他所有词的内积之和，并做Softmax激活。这样feed propagation，back propagation都要计算 $O$(词典大小) 时间复杂度。引入负样本的思想，以窗口为界，中心词和窗口内词为正样本，中心词和窗口外词为负样本，只要计算和求导只要更新少量词的向量即可。

  负采样改变了目标函数，正、负样本分别做词向量内积，用Sigmoid激活之后求积。有点类似Logistic Regression里面交叉熵损失函数，去掉指数后的乘积。

    1. trick：按窗口大小随机取s = [1, window_size]，正样本一次取s个，让模型更关注距离中心词更近的词。
    2. 负样本选取：词出现频数的0.75 次幂作为权重，一个正样本，对词典随机选取K个负样本。减弱频次差异的影响，让小概率词被采样概率变大。
    

#### 推导

每个词被表示成两个$d$维向量，词在词典中索引为$i$，当它为中心词时向量表示为$\boldsymbol{v}_i\in\mathbb{R}^d$，而为背景词时向量表示为$\boldsymbol{u}_i\in\mathbb{R}^d$。词典索引集$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$。

设中心词$w_c$在词典中索引为$c$，背景词$w_o$在词典中索引为$o$，给定中心词生成背景词的条件概率：
$$P(w_o \mid w_c) = \frac{\text{exp}(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}$$

给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$。假设给定中心词的情况下背景词的生成相互独立，当背景窗口大小为$m$时，最大似然函数即给定任一中心词生成所有背景词的概率：
$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)})$$

等价于最小化损失函数：
$$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, P(w^{(t+j)} \mid w^{(t)})$$

随机梯度下降，每次迭代随机采样较短子序列，计算该子序列的损失，然后计算梯度更新模型参数。梯度计算的关键是，条件概率的对数，对中心词向量和背景词向量的梯度。
$$\log P(w_o \mid w_c) =
\boldsymbol{u}_o^\top \boldsymbol{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)\right)$$

$$\begin{aligned}
\frac{\partial \text{log}\, P(w_o \mid w_c)}{\partial \boldsymbol{v}_c} 
&= \boldsymbol{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\boldsymbol{u}_j^\top \boldsymbol{v}_c)\boldsymbol{u}_j}{\sum_{i \in \mathcal{V}} \exp(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\\
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\boldsymbol{u}_j^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\right) \boldsymbol{u}_j\\ 
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \boldsymbol{u}_j
\end{aligned}
$$

训练结束后，对于词典中的任一索引为$i$的词，我们均得到该词作为中心词和背景词的两组词向量$\boldsymbol{v}_i$和$\boldsymbol{u}_i$。在自然语言处理应用中，一般使用中心词向量作为词的表征向量。

对中心词$w_c$，计算$\boldsymbol{v}_c$的梯度，需要词典中所有以$w_c$为中心词的条件概率，每一步梯度计算时间复杂度是$\boldsymbol{O}(|\mathcal{V}|)$，引入负采样解决此问题。